<h1>HW 3</h1>

Alan Wu

### imports

In [1]:
import pandas as pd 
import numpy as np 
import seaborn as sns 
import matplotlib.pyplot as plt

### load data

In [5]:
bmo_data = pd.read_excel('./data/BMO.xlsx')
bns_data = pd.read_excel('./data/BNS.xlsx')
cm_data = pd.read_excel('./data/CM.xlsx')
na_data = pd.read_excel('./data/NA.xlsx')
ry_data = pd.read_excel('./data/RY.xlsx')
td_data = pd.read_excel('./data/TD.xlsx')

bmo_data.head()


,date,open,high,low,close,adjusted_close,volume
0,2018-04-18,94.84,95.60,94.75,94.99,66.0176,1142387
1,2018-04-19,95.11,95.25,94.73,95.01,66.0315,954509
2,2018-04-20,95.10,95.94,95.10,95.57,66.4207,887700
3,2018-04-23,95.81,96.41,95.77,96.17,66.8377,959144
4,2018-04-24,96.49,97.16,96.34,96.68,67.1922,940562


In [16]:
data =  {'bmo':bmo_data, 'bns':bns_data, 'cm':cm_data, 'na':na_data, 'ry':ry_data, 'td':td_data}


### 1a

In [17]:
prices = []
for name, dataset in data.items(): 
    price = dataset[dataset['date'] == '2020-04-17']['close'] 
    prices.append(price.values[0])
    print(f"The closing price of {name} is {price.values[0]}")


The closing price of bmo is 69.99
The closing price of bns is 54.45
The closing price of cm is 80.9
The closing price of na is 54.36
The closing price of ry is 87.47
The closing price of td is 56.92


In [18]:
initial_shares_map = {}
for i,price in enumerate(prices): 
    initial_shares_map[data_names[i]] = 2_500_000 / prices[i]
    print(f"The initial shares of {data_names[i]} is {2_500_000 / prices[i]}")


The initial shares of bmo is 35719.38848406915
The initial shares of bns is 45913.682277318636
The initial shares of cm is 30902.348578491965
The initial shares of na is 45989.6983075791
The initial shares of ry is 28581.227849548417
The initial shares of td is 43921.29304286718


### 1b

To compute the volaility of the portfolio, we need to 

1. compute the value of the portfolio on each day leading up to the initial day 
1. compute the returns of the portfolio on each day leading up to the initial day 
1. compute the annualized volatility of the portfolio 
1. account for divididends

In [28]:
portfolio_df = pd.DataFrame(index = bmo_data.index)

In [30]:
#compute the portfolio value for each day 
for name, dataset in data.items(): 
    bank_value = dataset['adjusted_close'] * initial_shares_map[name]
    portfolio_df[name] = bank_value 

portfolio_df['Value'] = portfolio_df.sum(axis=1) 
portfolio_df = portfolio_df.drop(columns=['bmo', 'bns', 'cm', 'na', 'ry', 'td'])
portfolio_df['date'] = bmo_data['date'] 



In [31]:
portfolio_df.head()

,Value,date
0,2.441589e+07,2018-04-18
1,2.432648e+07,2018-04-19
2,2.444948e+07,2018-04-20
3,2.461406e+07,2018-04-23
4,2.464662e+07,2018-04-24


In [40]:
# Calculate log returns: ln(Pt / Pt-1)
# which is mathematically equivalent to np.log(price) - np.log(price_prev)
log_returns = np.log(portfolio_df['Value'] / portfolio_df['Value'].shift(1)).to_frame()

# If you need to keep the date column
if 'date' in portfolio_df.columns:
    log_returns['date'] = portfolio_df['date']

# Drop the first NaN value caused by the shift
log_returns = log_returns.dropna()

# Compute volatility
daily_vol = log_returns['Value'].std()
annual_vol = daily_vol * np.sqrt(252)


print(f"Annualized Volatility (Log Returns): {annual_vol}")

Annualized Volatility (Log Returns): 0.27204329894794826


### 1c

We need to estimate the value of the ADS with CAD 100 principal using monte carlo simulation. 

In [47]:
sigma = annual_vol 
s_0 = 100 
r = 0.0045 
q = 0.058
T = 5 
paths = 1000 
alpha = 0.95 
dt = 1.0

In [54]:
np.random.seed(42)

Z = np.random.normal(size=(paths,T))
drift = (r - q - 0.5 * sigma**2) * dt
vol_term = sigma * np.sqrt(dt)

S = np.zeros((paths, T))
current_S = np.full(paths, s_0)

for t in range(T):
    # Update current_S for the next step (Year t+1)
    current_S = current_S * np.exp(drift + vol_term * Z[:, t])
    S[:, t] = current_S

payoffs_PV = np.zeros(paths)
returns_list = np.zeros(paths)

#simulations
for i in range(paths):
    payoff_time = 0
    cash_flow = 0
    terminated = False
    
    # Check Autocall for Years 1 to 4
    for t_idx in range(4):
        year = t_idx + 1
        St = S[i, t_idx]
        
        if St >= s_0:
            # Autocall Event
            fixed_coupon = 16.0 * year
            
            # Participation
            # Threshold for participation is 16% * year
            pct_change_scaled = St - s_0# 100 * PctChange
            threshold_scaled = 16.0 * year
            
            bonus = 0.0
            if pct_change_scaled > threshold_scaled:
                bonus = 0.05 * (pct_change_scaled - threshold_scaled)
            
            cash_flow = 100.0 + fixed_coupon + bonus
            payoff_time = year
            terminated = True
            break
    
    # Check Maturity (Year 5) if not terminated
    if not terminated:
        year = 5
        St = S[i, 4]
        payoff_time = 5
        
        if St >= s_0:
            # Autocall at Maturity
            fixed_coupon = 16.0 * 5 # 80
            pct_change_scaled = St - s_0
            threshold_scaled = 16.0 * 5 # 80
            
            bonus = 0.0
            if pct_change_scaled > threshold_scaled:
                bonus = 0.05 * (pct_change_scaled - threshold_scaled)
            
            cash_flow = 100.0 + fixed_coupon + bonus
            
        else:
            # No Autocall
            # Check Barrier (70% of Initial)
            if St >= 0.70 * s_0:
                # Principal Protected
                cash_flow = 100.0
            else:
                # 1:1 Loss
                cash_flow = St # Since S0=100, Payoff = 100 * (St/100) = St
    
    # Discount to Present Value
    payoffs_PV[i] = cash_flow * np.exp(-r * payoff_time)
    
    # Calculate Equivalent Annually Compounded Rate of Return for Part (d)
    # (CashFlow / 100)^(1/Time) - 1
    # Note: If CashFlow is 0 (impossible here but good to check), return is -100%
    # If Time is 0 (impossible), handle error.
    ann_return = (cash_flow / 100.0)**(1.0 / payoff_time) - 1.0
    returns_list[i] = ann_return

In [55]:
#compute the expected value and the confidence interval for the expected value 
estimated_value = np.mean(payoffs_PV)
std_dev = np.std(payoffs_PV, ddof=1)
std_error = std_dev / np.sqrt(paths)
ci_lower = estimated_value - 1.96 * std_error
ci_upper = estimated_value + 1.96 * std_error

# Statistics for Part (d)
mean_return = np.mean(returns_list)
std_return = np.std(returns_list, ddof=1)

print(f"Estimated Value: {estimated_value}")
print(f"Standard Error: {std_error}")
print(f"95% CI: [{ci_lower}, {ci_upper}]")
print(f"Mean Return (Part d): {mean_return}")
print(f"Std Dev Return (Part d): {std_return}")

Estimated Value: 91.52754352551192
Standard Error: 1.3388041235423587
95% CI: [88.90348744336889, 94.15159960765494]
Mean Return (Part d): 0.02424485459934898
Std Dev Return (Part d): 0.16104074795777637
